In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import nengo

# Math

The impulse equation for the Gammatone filter is

$$g(t) = \frac{a t^{n - 1} \cos (2 \pi f_c t \phi)}{e^{2 \pi b t}}$$

where

* $a$ is a gain factor
* $n$ is the order of the filter
* $f_c$ is the characteristic frequency
* $\phi$ is the phase (ignored from this point onward -- has little effect one the filter)
* $b$ is the bandwidth (recommended setting: 1.019 times the ERB)

In [ ]:
# Impulse response for a cochlear channel
t = np.linspace(0, 0.015, 500)
n = 4  # order of 4
b = 125  # bandwidth of 125Hz
fc = 1000  # centered at 1000Hz
plt.plot(t, t ** (n-1) * np.exp(-2 * np.pi * b * t) * np.cos(2 * np.pi * fc * t))

In [ ]:
# Gold standard I'm using: Brian's implementation
from brian import *
from brian.hears import *
import matplotlib.pyplot as plt

sound = whitenoise(100*ms).ramp()
sound.level = 50*dB

nbr_center_frequencies = 40
b1 = 1.019  #factor determining the time constant of the filters
#center frequencies with a spacing following an ERB scale
center_frequencies = erbspace(80*Hz, 1000*Hz, nbr_center_frequencies)
gammatone = Gammatone(sound, center_frequencies, b=b1)

gt_mon = gammatone.process()

plt.plot(sound)
plt.figure()

plt.imshow(gt_mon.T, aspect='auto', origin='lower left',
       extent=(0, sound.duration/ms,
               center_frequencies[0], center_frequencies[-1]))
plt.yscale('log')
plt.title('Cochleogram')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (ms)')

In [ ]:
fs = 44100.
dt = 1. / fs
cf = erbspace(165, 1000, 40)
fw, fb = gammatone_coeffs(cf, fs)
gt = gammatone(sound.ravel(), fw, fb, lfilter_df1)
#gt = gammatone(sound.ravel(), fw, fb, lfilter_tdf2)

plt.plot(sound)
plt.figure()
plt.imshow(gt, aspect='auto', origin='lower left',
           extent=(0, sound.size/fs, cf[0], cf[-1]))
plt.yscale('log')
plt.title('Cochleogram')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')

In [ ]:
# General LTI step function using transposed Direct-II form.
# Unfortunately, this doesn't fix the issue I'm having with
# some frequencies blowing up...

class GeneralD2(nengo.synapses.LinearFilter.Step):
    def __init__(self, num, den, output):
        order = max(len(num), len(den) + 1)
        if num.ndim == 1:
            assert den.ndim == 1
            self._num = np.zeros(order)
            self._den = np.zeros(order)
            self.z = np.zeros(order - 1)
        elif num.ndim == 2:
            assert den.ndim == 2
            self._num = np.zeros((order, num.shape[1]))
            self._den = np.zeros((order, den.shape[1]))
            self.z = np.zeros((order - 1, num.shape[1]))
        self._num[:len(num)] = num
        self._den[0] = 1.0  # we take this out for General but put it back in here
        self._den[1:] = den
        super(GeneralD2, self).__init__(num, den, output)

    def __call__(self, signal):
        self.output[...] = self._num[0] * signal + self.z[0]
        
        self.z = np.vstack((self.z[1:],
                            np.zeros((1, self.z.shape[1]))))
        self.z = self.z + self._num[1:] * signal - self._den[1:] * self.output

In [ ]:
import collections
import functools

def erbspace(low, high, n_samples, ear_q=9.26449, min_bw=24.7, order=1):
    """Returns the centre frequencies on an ERB scale.
    
    ``low``, ``high``
        Lower and upper frequencies
    ``N``
        Number of channels
    ``earQ=9.26449``, ``minBW=24.7``, ``order=1``
        Default Glasberg and Moore parameters.
    """
    low = float(low)
    high = float(high)
    qbw = ear_q * min_bw
    cf = (-qbw + np.exp(np.arange(n_samples)
                        * (-np.log(high + qbw) + np.log(low + qbw))
                        / (n_samples - 1))
          * (high + qbw))
    cf = cf[::-1]
    return cf

def syn2process(syn_step):
    def _call(t, x):
        syn_step(x)
        return syn_step.output
    return _call




class Gammatone(nengo.processes.Process):
    def __init__(self, freq, b=1.019, ear_q=9.26449, min_bw=24.7, erb_order=1.):
        self.freq = freq
        self.b = b
        self.ear_q = ear_q
        self.min_bw = min_bw
        self.erb_order = erb_order

    def make_step(self, size_in, size_out, dt, rng):
        assert size_in == 1
        assert size_out == self.freq.size

        freq = np.atleast_1d(self.freq)
        erb = (((freq / self.ear_q) ** self.erb_order + self.min_bw ** self.erb_order)
               ** (1. / self.erb_order))
        bandwidth =  2 * self.b * np.pi * erb

        # Cache the results of computing parts of the below computations
        fp = freq * np.pi * dt
        bw = bandwidth * dt
        ebt = {1: np.exp(bw),
               2: np.exp(2 * bw),
               3: np.exp(3 * bw),
               4: np.exp(4 * bw),
               5: np.exp(5 * bw),
               6: np.exp(6 * bw),
               7: np.exp(7 * bw),
               -8: np.exp(-8 * bw)}
        e4jfp = np.exp(4j * fp)
        g = -2 * e4jfp * dt + 2 * np.exp(-bw + 2j * fp) * dt
        cosf = {2: np.cos(2 * fp),
                4: np.cos(4 * fp),
                6: np.cos(6 * fp),
                8: np.cos(8 * fp)}
        sinf = {2: np.sin(2 * fp)}
        z_pos = np.sqrt(3 + 2 ** 1.5)
        z_neg = np.sqrt(3 - 2 ** 1.5)
        g1 = g * (cosf[2] - z_neg * sinf[2])
        g2 = g * (cosf[2] + z_neg * sinf[2])
        g3 = g * (cosf[2] - z_pos * sinf[2])
        g4 = g * (cosf[2] + z_pos * sinf[2])

        gain = np.abs(g1 * g2 * g3 * g4
                      / (-2 / ebt[2] - 2 * e4jfp + 2 * (1 + e4jfp) / ebt[1])
                      ** 4)

        if freq.size > 1:
            num = np.zeros((5, freq.size))
            den = np.zeros((8, freq.size))
        else:
            num = np.zeros(5)
            den = np.zeros(8)

        numgain = dt ** 4 / gain
        num[0] = numgain
        num[1] = numgain * -4 * cosf[2] / ebt[1]
        num[2] = numgain *  6 * cosf[4] / ebt[2]
        num[3] = numgain * -4 * cosf[6] / ebt[3]
        num[4] = numgain *      cosf[8] / ebt[4]

        den1 = -8 * cosf[2]
        den26 = 4 * (4 +  3 * cosf[4])
        den35 = -8 * (6 * cosf[2] + cosf[6])
        den4 = 2 * (18 + 16 * cosf[4] + cosf[8])
        den7 = -8 * cosf[2]
        # Remove the initial np.ones because of how we do filtering
        den[0] = den1  / ebt[1]
        den[1] = den26 / ebt[2]
        den[2] = den35 / ebt[3]
        den[3] = den4  / ebt[4]
        den[4] = den35 / ebt[5]
        den[5] = den26 / ebt[6]
        den[6] = den7  / ebt[7]
        den[7] = ebt[-8]

        output = np.zeros(self.freq.size)
        #return syn2process(GeneralD2(num, den, output))
        return syn2process(nengo.synapses.LinearFilter.General(num, den, output))

In [ ]:
# Simple Nengo model that does the filtering during a sim
fs = 44100.
dt = 1. / fs
#freq = erbspace(50, 150, 100)
freq = np.array([1])

rng = np.random.RandomState(10)
#rng = None  # Don't seed

with nengo.Network() as net:
    wnoise = nengo.processes.WhiteNoise(nengo.dists.Gaussian(mean=0, std=1.))
    if rng is not None:
        audio = nengo.Node(output=wnoise.make_step(0, 1, 0.001, rng))
    else:
        audio = nengo.Node(output=wnoise)
    # just kidding, sine wave
    audio.output = lambda t: np.sin(2 * np.pi * t * 100)
    # or maybe constant?
    audio.output = np.array([0.001])
    gt = nengo.Node(output=Gammatone(freq), size_in=1, size_out=freq.size)
    nengo.Connection(audio, gt)
    audio_p = nengo.Probe(audio, synapse=None)
    gt_p = nengo.Probe(gt, synapse=None)

sim = nengo.Simulator(net, dt=dt)
sim.run(0.032)

sound = sim.data[audio_p]
plt.plot(sim.trange(), sound)
plt.xlim(right=sim.trange()[-1])
plt.figure()
if freq.size > 1:
    plt.imshow(sim.data[gt_p].T, aspect='auto', origin='lower left',
               extent=(0, sound.size/fs, freq[0], freq[-1]))
    #plt.yscale('log')
    plt.ylabel('Frequency (Hz)')
else:
    plt.plot(sim.trange(), sim.data[gt_p])
    plt.xlim(right=sim.trange()[-1])
plt.title('Cochleogram')
plt.xlabel('Time (s)')
print(sim.data[gt_p].T.min(), sim.data[gt_p].T.max())